In [74]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re
import pandas as pd
import subprocess
import time
from tqdm import tqdm

In [62]:
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)

In [76]:
df = pd.read_csv("output.txt")
for ref in list(df["ref"])[:3]:
    url = "https://www.kaggle.com/code/" + ref
    print(url)

https://www.kaggle.com/code/adhamtarek147/cifar-10
https://www.kaggle.com/code/matinmahmoudi/from-scratch-linear-regression-using-oop
https://www.kaggle.com/code/abhishek0032/data-science-toolkit-codes-skills-to-succeed


In [73]:
# TEST RUN
def extract_metrics(url):
    #url = 'https://www.kaggle.com/code/m1nty03/rice-type-classification'
    name = url.split("www.kaggle.com/code/")
    name = name[-1].replace("/", "--")
    driver.get(url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'rendered-kernel-content'))
    )
    
    iframe = driver.find_element(By.ID, 'rendered-kernel-content')
    iframe_src = iframe.get_attribute('src')
    response = requests.get(iframe_src)
    soup = bs(response.text, 'html.parser')
    
    pattern1 = r'Epoch\s*(\d+)/\d+.*? - accuracy:\s*(\d+\.\d+)\s*- loss:\s*(\d+\.\d+)'
    pattern2 = r'Epoch\s*(\d+)/\d+.*? - accuracy:\s*(\d+\.\d+)\s*- loss:\s*(\d+\.\d+)(?:.*? - val_accuracy:\s*(\d+\.\d+)\s*- val_loss:\s*(\d+\.\d+))?'
    matches = re.findall(pattern2, soup.text, re.DOTALL)
    
    structured_data = []
    for match in matches:
        epoch = int(match[0])
        acc = float(match[1])
        loss = float(match[2])
        val_acc = float(match[3]) if match[3] else None
        val_loss = float(match[4]) if match[4] else None
        structured_data.append((epoch, acc, loss, val_acc, val_loss))
    
    # Create a DataFrame for better readability
    df = pd.DataFrame(structured_data, columns=['Epoch', 'Accuracy', 'Loss', 'Val_Accuracy', 'Val_Loss'])
    if len(df) > 0:
        print(f"found {len(df)} metrics for {name}")
        df.to_csv(name + ".csv", index=False)
        return df
    else:
        return

df = extract_metrics("https://www.kaggle.com/code/adhamtarek147/cifar-10")


In [ ]:
for ref in tqdm(list(df["ref"])):
    url = "https://www.kaggle.com/code/" + ref
    #print(f"processing url {url}")
    extract_metrics(url)

  0%|▎                                                                              | 5/1070 [00:19<1:08:02,  3.83s/it]

found 20 metrics for junaid512--real-vs-fake-img


  1%|▊                                                                             | 12/1070 [00:44<1:02:37,  3.55s/it]

In [48]:
driver.quit()